<a href="https://colab.research.google.com/github/karaage0703/edge-ai-cv/blob/main/image_classification/image_classification_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像分類（PyTorch版）



## 教師データのダウンロード

ジャンケンの手の形の教師データをGitHubからダウンロード（Clone）します。

2,3行目はダウンロードしたデータから、使用するデータ以外の不要なファイルを削除しています。

教師データをダウンロードして、不要なファイルを削除します。

In [ ]:
!git clone https://github.com/karaage0703/janken_dataset original_datasets
!rm -rf /content/original_datasets/.git
!rm /content/original_datasets/LICENSE

データの中身の確認

In [ ]:
!ls original_datasets

In [ ]:
!ls original_datasets/choki

In [ ]:
from IPython.display import Image as IPImage
from IPython.display import display_jpeg
display_jpeg(IPImage('original_datasets/choki/choki_01.jpg'))

## 教師データを訓練データ（Train Data）とテストデータ（Validation Data）に分ける

ディレクトリの構造を可視化するための'tree'というソフトをインストールします。

In [ ]:
!apt-get -qq install tree

In [ ]:
!tree -d /content/original_datasets

教師データのディレクトリと、ターゲットとなるディレクトリ（この下に訓練データのディレクトリと検証データのディレクトリが生成される）を指定。

In [ ]:
dataset_original_dir = '/content/original_datasets'
dataset_root_dir = '/content/datasets'

In [ ]:
!wget https://raw.githubusercontent.com/karaage0703/karaage-ai-book/master/util/split_train_val.py

In [ ]:
import split_train_val

In [ ]:
split_train_val.image_dir_train_val_split(dataset_original_dir, dataset_root_dir, train_size=0.66)

In [ ]:
!tree -d /content/datasets

In [ ]:
train_dir = '/content/datasets/train'
val_dir = '/content/datasets/val'

### データのロード

必要なライブラリをインポートします

PyTorchとデータの前処理や可視化をしてくれるtorchvisionという便利なライブラリをインポートします。

In [ ]:
import torch
from torchvision import transforms, datasets
import matplotlib.pyplot as plt

ImageFolderを使って、訓練ディレクトリの画像をdataset_train、検証ディレクトリの画像を dataset_valとして読み込みます

In [ ]:
dataset_train = datasets.ImageFolder(root=train_dir)
dataset_val = datasets.ImageFolder(root=val_dir)

dataset_trainとdataset_valの中身を確認します。それぞれ 109と58のデータが格納されていることが分かります。

In [ ]:
print(dataset_train)
print(dataset_val)

dataset_train[0]〜dataset_train[108]の中身は、PIL形式の画像データとラベルのインデックスが格納されています。

In [ ]:
x, y = dataset_train[0]
print(x)
print(y)
x, y = dataset_val[0]
print(x)
print(y)

具体的な中身は`__getitem__`で確認できます。

最後の数字はラベルを示しています。最初に表示させたディレクトリの表示順となり以下となります。

```
0: choki
1: gu
2: pa
```

In [ ]:
print(dataset_train.__getitem__(0))
print(dataset_train.__getitem__(50))
print(dataset_train.__getitem__(100))

matplotlibで中身を確認してみましょう。

In [ ]:
image_numb = 6 # 3の倍数を指定してください
for i in range(0, image_numb):
  ax = plt.subplot(int(image_numb / 2), 3, i + 1)
  plt.tight_layout()
  ax.set_title(str(i))
  plt.imshow(dataset_train[i][0])

datasetは使えるようになりましたが、PyTorchで扱うためにサイズの変換と、テンソル化というPyTorch等のディープラーニングのフレームワークで多く使われる、Tensor形式に変換する必要があります。

具体的には、transforms機能を用いて、以下の前処理を実施します。
- リサイズ(transforms.Resize())
- テンソル化(transforms.ToTensor())

以下のように書くと、前処理を実施したデータが読み込めます

参考： https://qiita.com/kazetof/items/6a72926b9f8cd44c218e

In [ ]:
IMAGE_SIZE = 64
data_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor()
])

dataset_train = datasets.ImageFolder(root=train_dir, transform=data_transform)
dataset_val = datasets.ImageFolder(root=val_dir, transform=data_transform)

中身を確認します。前処理情報が追記されています。

In [ ]:
print(dataset_train)
print(dataset_val)

生の値を確認しましょう。教師データのPIL形式の画像が、テンソル形式に変換され、値の範囲が0〜1になっていることが分かります。

In [ ]:
print(dataset_train.__getitem__(0))

また、画像のサイズを確認しておきましょう。

In [ ]:
print(dataset_train[0][0].shape)

この配列は画像のサイズを示しています。最初の3はチャンネル数です。RGB形式なので(Red, Green, Blue）の3次元となります。。

そのあとの、64は先ほど示した画像のサイズを示しています。PyTorchでは最初にチャンネル数を指定することになるので、注意して下さい。

続いて、この中身をmatplotlibで画像として確認しましょう。ただし、datasetはテンソル化されているので、datasetをnumpy()でNumpy配列とする必要があります。

また、PyTorchはChannel Firstという、チャンネル（RGB画像の場合はR,G,Bの3チャンネルなので3）を一番最初に並べる形式のため、画像として表示するため、Numpyのtransposeでチャンネルを最後に並び変えます。

具体的には `dataset`の後ろに `.numpy().transpose((1, 2, 0))`を追加します。

ちゃんと表示されました。図の軸の数字から、サイズが64x64に変換されていることも分かります。

In [ ]:
image_numb = 6 # 3の倍数を指定してください
for i in range(0, image_numb):
  ax = plt.subplot(int(image_numb / 2), 3, i + 1)
  plt.tight_layout()
  ax.set_title(str(i))
  plt.imshow(dataset_train[i][0].numpy().transpose((1, 2, 0)))

## ラベルファイルの作成

学習するファイルのラベルを作成します

必要なライブラリをインポートします

In [ ]:
import sys
import os
import shutil

データを保存する場所を指定します。

今後、ラベルデータやモデルデータなどは以下のディレクトリに保存されます。

In [ ]:
backup_dir = './model'

ラベルデータを作成します（最後に表示される class numberが画像の種類の数です）

In [ ]:
labels = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
labels.sort()

if os.path.exists(backup_dir):
    shutil.rmtree(backup_dir)

os.makedirs(backup_dir)

with open(backup_dir + '/labels.txt','w') as f:
    for label in labels:
        f.write(label+"\n")

NUM_CLASSES = len(labels)
print("class number=" + str(NUM_CLASSES))

ラベルを確認します。ラベル名（choki, gu, pa）が並んでいればOKです

In [ ]:
!cat ./model/labels.txt

## モデルの作成

ハイパーパラメータを設定します。

TensorFlow（Keras）のときとは学習率が大きく異なるため注意して下さい。

In [ ]:
# 学習率
LEARNING_RATE = 1.0
# エポック（世代数）
EPOCHS = 20
# バッチサイズ
BATCH_SIZE = 4

学習用のデータをデータセットからロードするデータローダーを作成します。これにより、データセットからバッチと呼ばれるデータのまとまりで、データをロードすることができます。

- batch_size: バッチサイズ
- shuffle: シャッフルするか
- num_workers: データをロードするコア数

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset_train,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True,
                                          num_workers=1)
test_dataloader = torch.utils.data.DataLoader(dataset_val,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True,
                                          num_workers=1)

モデルを作成します。ポイントとなる1層目の畳み込み層のConv2dの定義は以下です。
```
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
```

今回扱うRGB画像は、入力チャンネルがR,G,Bの3チャンネル。出力チャンネルは64, カーネルサイズは 3x3、padding=1とします。paddingを1としているのは、畳み込み前後で画像のサイズを変えないためです。

具体的には以下となります。

```
nn.Conv2d(3, 32, (3, 3), 1, 1)
```

またfc層への接続は、注意が必要です。

fc層(fc1) の入力は以下となります。

```
チャネル数 * イメージマップのサイズ
```

チャネル数は、conv2層の出力 64 となります。また、イメージマップのサイズは、最初 64x64だったものが、CNN層、プーリング層によりサイズが変わるため計算が必要です。

CNNの場合は、カーネルサイズ、パディングによって変わりますが、今回は入力と出力でサイズが変わらないように調整してあります。

マックスプーリングは、2x2で実施しているので半分になります。よって 32x32 がイメージマップのサイズとなります。

全結合層へ繋げるための1次元へ展開は、x.viewを使います。-1を引数とすることで、self.num_flat_features(x)の値から自動的に値が決まります。

```
x = x.view(-1, self.num_flat_features(x))
```

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, (3, 3), 1, 1)
    self.conv2 = nn.Conv2d(32, 64, (3, 3), 1, 1)
    self.dropout1 = nn.Dropout(0.25)
    self.dropout2 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(64 * 32 * 32, 128) # 32 = 64(IMAGE_SIZE) / 2
    self.fc2 = nn.Linear(128, NUM_CLASSES)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output

モデルを定義して、確認します。

最初の1行は、GPU(CUDA)が使えるかを判断しています。

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model().to(device)
print(model)

パラメータの確認をします

In [ ]:
#model.parameters()をリストに型変換することでパラメータを取り出せる
params=list(model.parameters())
#len(params)はパラメータの種類の数
print(len(params))
#conv1の重みのサイズを確認する
print(params[0].size())
#conv2の重みのサイズを確認する
print(params[2].size())

ニューラルネットワークのテスト。ニューラルネットの入力に合わせた、教師データと同じサイズのランダムな画像をネットワークに入力して出力を確認します。

`個数, チャネル数, 画像サイズ, 画像サイズ`

デバイス(GPU or CPU)に応じた変換(`.to(device)`)が必要なことに注意しましょう。

最終的な出力が、3つ（ラベルの数）になっていることを確認します。

In [ ]:
input = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE)
input = input.to(device)
out = model(input)
print(out)

最適化方法を定義

In [ ]:
from torch import optim
optimizer = optim.Adadelta(model.parameters(), lr=LEARNING_RATE)

AIモデルの学習を行います

In [ ]:
def train(model, device, train_dataloader, optimizer):
  train_loss = 0

  model.train()
  for batch_idx, (data, target) in enumerate(train_dataloader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    train_loss += loss.item()

    loss.backward()
    optimizer.step()

  train_loss /= len(train_dataloader)

  return train_loss

In [ ]:
def test(model, device, test_dataloader):
  model.eval()
  val_loss = 0
  val_acc = 0
  correct = 0
  total = 0

  with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_dataloader):
      data, target = data.to(device), target.to(device)
      output = model(data)
      val_loss += F.nll_loss(output, target).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
      total += target.size(0)

  val_loss = val_loss / len(test_dataloader)
  val_acc = correct / total

  return val_loss, val_acc

In [ ]:
train_loss_list = []

val_loss_list = []
val_acc_list = []

for epoch in range(1, EPOCHS + 1):
  train_loss = train(model, device, train_dataloader, optimizer)
  val_loss, val_acc = test(model, device, test_dataloader)

  train_loss_list.append(train_loss)
  val_loss_list.append(val_loss)
  val_acc_list.append(val_acc)

  print('epoch: {:d}'.format(epoch))
  print('val_loss: {:.4f}, val_acc: {:.4f}'.format(100. * val_loss, val_acc))

## 学習結果の可視化

lossを確認します。lossは正解との差を意味するモデルを評価するための指標で、低いほど良い値となります。

AIモデルは、この値が低くなるように学習を進めます。

In [ ]:
plt.plot(train_loss_list)
plt.plot(val_loss_list)
plt.title('Training and validation loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.xlim([0.0, EPOCHS])
plt.legend(['loss', 'val_loss'], loc='lower right')
plt.show()

acc（精度）を確認します。accが訓練データでの精度で、この値が高いほど良い性能を意味します。
例えば0.5だと50%の正解率ということになります。

val_accというのが訓練に使っていないテストデータを使っての精度です。  
いわゆる、本当の精度と言われるものは、val_accの方となります。

In [ ]:
plt.plot(val_acc_list)
plt.title('Validation acc')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.xlim([0.0, EPOCHS])
plt.ylim([0.0, 1.0])
plt.legend(['val_loss'], loc='lower right')
plt.show()

## 学習させたモデルを使った推定

学習させたモデルを使って、画像の推定を行います

In [ ]:
def visualize_model(model, num_images=6):
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    fig = plt.figure(figsize=(10, num_images))

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_dataloader):
            data, target = data.to(device), target.to(device)

            output = model(data)
            _, pred = torch.max(output, 1)

            for n in range(data.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//3, 3, images_so_far)
                ax.axis('off')
                color = 'green' if pred[n] == target[n] else 'red'
                ax.set_title('predicted: {} , label: {}'.format(labels[pred[n]], labels[target[n]]), color=color)
                plt.imshow(data.cpu().data[n].numpy().transpose((1, 2, 0)))
                if images_so_far == num_images:
                    return

In [ ]:
visualize_model(model, num_images=12)

## 混合行列(Confusion Matrix)の可視化

https://stackoverflow.com/questions/53290306/confusion-matrix-and-test-accuracy-for-pytorch-transfer-learning-tutorial

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

model.eval()

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0, dtype=torch.long, device='cpu')
lbllist=torch.zeros(0, dtype=torch.long, device='cpu')

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_dataloader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, pred = torch.max(output, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,pred.view(-1).cpu()])
        lbllist=torch.cat([lbllist,target.view(-1).cpu()])

# Confusion matrix
cm = confusion_matrix(lbllist.numpy(), predlist.numpy())
cm = cm/cm.sum(1)

sns.heatmap(cm, annot=True, square=True, cmap=plt.cm.Blues,
            xticklabels=labels,
            yticklabels=labels)

plt.title("Confusion Matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.xlim([0.0, NUM_CLASSES])
plt.ylim([0.0, NUM_CLASSES])
plt.show()

## 学習モデルの保存とダウンロード

学習モデルを保存します。また、Google Colaboratory上のファイルは、自動的に消えてしまうのでモデルをローカルにダウンロードします。

最初にモデルを保存します。

In [ ]:
model_path = 'janken.pth'
torch.save(model, model_path)

モデルをダウンロードします。

In [ ]:
from google.colab import files
files.download(model_path)

## TorchモデルからONNXモデルへの変換

In [ ]:
onnx_path = 'janken.onnx'

# Input size(N, C, H, W)
x = torch.randn(1, 3, 64, 64)
x = x.to(device)

torch.onnx.export(
    model,                                         # model
    x,                                              # input data
    onnx_path,                                  # ONNX file name
    opset_version=11,                               # ONNX version
    )

In [ ]:
from google.colab import files
files.download(onnx_path)

## まとめ

PyTorchの学習から推論とモデルの保存ができました。

## 参考リンク

以下は多くを参考にした情報です。

前処理（データセット・データローダー）
- https://sonaeru-blog.com/pytorch-dataset/
- http://kaga100man.com/2019/01/09/post-89/
- https://qiita.com/takurooo/items/e4c91c5d78059f92e76d
- https://discuss.pytorch.org/t/questions-about-imagefolder/774/6

自作データセットの学習
- http://robonchu.hatenablog.com/entry/2017/10/23/173317

ニューラルネットワーク・学習
- https://github.com/pytorch/examples/blob/master/mnist/main.py
- http://aidiary.hatenablog.com/entry/20180205/1517832760
- https://pytorch.org/docs/stable/nn.html
- https://blog.shikoan.com/pytorch-convtranspose2d/
- https://www.hellocybernetics.tech/entry/2017/10/20/025702
- https://www.hellocybernetics.tech/entry/2018/02/20/182906
- https://www.sambaiz.net/article/205/
- https://www.procrasist.com/entry/19-pytorch
- https://qiita.com/mckeeeen/items/e255b4ac1efba88d0ca1
- https://tips-memo.com/python-pytorch-3
- https://qiita.com/kamata1729/items/7adaead883566e3043b5

認識結果の可視化
- http://torch.classcat.com/2018/04/29/pytorch-tutorial-transfer-learning/

つまづきポイント
- https://qiita.com/takurooo/items/e356dfdeec768d8f7146

モデル変換
- https://imagingsolution.net/deep-learning/pytorch/save_pytorch_model_onnx_csharp/

## 参考書籍
つくりながら学ぶ！PyTorchによる発展ディープラーニング